<a href="https://colab.research.google.com/github/aidyai/vLM/blob/main/vLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/BAAI-DCAI/Bunny.git

Cloning into 'Bunny'...
remote: Enumerating objects: 247, done.
remote: Counting objects: 100% (86/86), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 247 (delta 38), reused 43 (delta 17), pack-reused 161
Receiving objects: 100% (247/247), 24.51 MiB | 20.26 MiB/s, done.
Resolving deltas: 100% (75/75), done.


In [ ]:
!pip install accelerate

In [ ]:
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer
from PIL import Image
import warnings

In [ ]:
# disable some warnings
transformers.logging.set_verbosity_error()
transformers.logging.disable_progress_bar()
warnings.filterwarnings('ignore')

# set device
torch.set_default_device('cuda')  # or 'cuda'

In [ ]:
# create model
model = AutoModelForCausalLM.from_pretrained(
    'BAAI/bunny-phi-2-siglip',
    torch_dtype=torch.float16,
    device_map='auto',
    trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(
    'BAAI/bunny-phi-2-siglip',
    trust_remote_code=True)

In [ ]:
# text prompt
prompt = "please give me the html and css code for this UI"
text = f"<image>\n{prompt} ASSISTANT:"
text_chunks = [tokenizer(chunk).input_ids for chunk in text.split('<image>')]
input_ids = torch.tensor(text_chunks[0] + [-200] + text_chunks[1], dtype=torch.long).unsqueeze(0)


In [ ]:
# image, sample images can be found in https://huggingface.co/BAAI/bunny-phi-2-siglip/tree/main/images
image = Image.open("/content/UI DESIGN.jpg")
image_tensor = model.process_images([image], model.config).to(dtype=model.dtype)


In [ ]:
torch.cuda.empty_cache()
import gc
gc.collect()

0

In [ ]:
# generate
output_ids = model.generate(
    input_ids,
    images=image_tensor,
    max_new_tokens=100,
    use_cache=True)[0]

print(tokenizer.decode(output_ids[input_ids.shape[1]:], skip_special_tokens=True).strip())

I'm sorry, but I cannot provide the HTML and CSS code for this UI as it is not possible to see the exact design and layout of the app. However, I can provide you with some general tips on how to create a visually appealing and user-friendly app interface.


In [ ]:
%cd /content

/content


In [ ]:
!rm -r ./llm-awq

In [ ]:
!git clone https://github.com/mit-han-lab/llm-awq.git

Cloning into 'llm-awq'...
remote: Enumerating objects: 719, done.
remote: Counting objects: 100% (356/356), done.
remote: Compressing objects: 100% (197/197), done.
remote: Total 719 (delta 205), reused 219 (delta 158), pack-reused 363
Receiving objects: 100% (719/719), 52.21 MiB | 24.90 MiB/s, done.
Resolving deltas: 100% (364/364), done.


In [ ]:
%cd /content/llm-awq

/content/llm-awq


In [ ]:
#!python -m pip install .
!python /content/llm-awq/awq/kernels/setup.py install

In [ ]:
import awq_inference_engine

In [ ]:
import torch
import numpy as np

from PIL import Image
import gc
import requests
from io import BytesIO

from transformers import AutoConfig, AutoTokenizer
from accelerate import load_checkpoint_and_dispatch

from awq.quantize.pre_quant import run_awq, apply_awq
from awq.quantize.quantizer import real_quantize_model_weight

from tinychat.utils.load_quant import load_awq_model
from tinychat.utils.tune import device_warmup, tune_all_wqlinears
from tinychat.utils.prompt_templates import get_prompter, get_stop_token_ids
from tinychat.utils.llava_image_processing import process_images, load_image
from tinychat.models.llava_llama import LlavaLlamaForCausalLM
from tinychat.stream_generators.llava_stream_gen import LlavaStreamGenerator
from tinychat.modules import make_quant_norm, make_quant_attn, make_fused_mlp, make_fused_vision_attn



In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# This demo only support single GPU for now

model_path = "/dataset/llava/llava-v1.5-7b"  # Please change here
quant_path = "../quant_cache/llava-v1.5-7b-w4-g128-awq.pt"  # place to dump quant weights

device = "cuda"

In [ ]:
!pip install autoawq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.8/76.8 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 85.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.1 MB/s eta 0:00:00


In [ ]:
import os
print(os.getcwd())

/content


In [ ]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer


model_path = 'llava-hf/llava-v1.6-mistral-7b-hf'
quant_path = './llava-awq'
quant_config = { "zero_point": True, "q_group_size": 128, "w_bit": 4, "version": "GEMM" }

# Load model
model = AutoAWQForCausalLM.from_pretrained(
    model_path, **{"low_cpu_mem_usage": True}
)
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

# Quantize
model.quantize(tokenizer, quant_config=quant_config)

# Save quantized model
model.save_quantized(quant_path)
tokenizer.save_pretrained(quant_path)

print(f'Model is quantized and saved at "{quant_path}"')

In [1]:
!git clone https://github.com/DLCV-BUAA/TinyLLaVABench.git
%cd TinyLLaVABench

Cloning into 'TinyLLaVABench'...
remote: Enumerating objects: 609, done.
remote: Counting objects: 100% (117/117), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 609 (delta 64), reused 55 (delta 27), pack-reused 492
Receiving objects: 100% (609/609), 1.96 MiB | 4.27 MiB/s, done.
Resolving deltas: 100% (252/252), done.
/content/TinyLLaVABench


In [ ]:
#!conda create -n tinyllava python=3.10 -y
#!conda activate tinyllava
!pip install --upgrade pip  # enable PEP 660 support
!pip install -e .

/bin/bash: line 1: conda: command not found
/bin/bash: line 1: conda: command not found
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
Obtaining file:///content/TinyLLaVABench
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.2/150.2 kB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 kB 10.9 MB/s eta 0:00:00
